In [1]:
#Importing all necessary libraries
!pip3 install nltk
!pip3 install pylanguagetool
!pip install pyspellchecker
!pip install pymysql
!pip install gensim
!pip install textblob
from spellchecker import SpellChecker
spell = SpellChecker()
import nltk
from textblob import TextBlob
from pylanguagetool import api
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords 
import pymysql
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from gensim.parsing.preprocessing import remove_stopwords,stem_text,strip_tags
from spellchecker import SpellChecker
import re

##############################################################################################################
#Defining the spelling and grammer check 

def grammer_spelling_check(data):
    sent=(sent_tokenize(data))
    c1=1
    for i in sent:
        c2=1
        word = (word_tokenize(i))
        for j in word:
            if(j=="."*len(j)):
                print("Repitition of punctuation in position "+str(c2) +" in line "+str(c1))
            else:
                if(len(j)==1 and ((j<'a' or j>'z') and (j<'A' or j>'Z')) and ((c2-1)!=0)):
                    if(word[c2-2]!="ouch" and word[c2-2]!="aah" and word[c2-2]!="aha" and word[c2-2]!="oho" and word[c2-2]!="hurray" and word[c2-2]!="alas" and word[c2-2]!="shit" and word[c2-2]!="hey" and word[c2-2]!="hi" and word[c2-2]!="huh"):
                        print("Position of punctuation is not justified in position "+str(c2) +" in line "+str(c1))
                    else:
                        if((c2-1)!=(len(word)-1)):
                            if(word[c2-1]==word[c2]):
                                print("Repeatition of punctuation in position "+str(c2) +" in line "+str(c1))
                else:
                    #eikhane jabotio ja ache dhokate hbe
                    if(spell.correction(j)!=j):
                        print("Possible spelling mistake in position "+str(c2)+" in line "+str(c1))#+" probable word "+spell.correction(j))
                    else:
                        if(TextBlob(j).correct()!=j):
                            print("Possible spelling mistake in position "+str(c2)+" in line "+str(c1))#+" probable word "+spell.correction(j))

            c2=c2+1
        check=api.check(i,api_url='https://languagetool.org/api/v2/',lang='en-US', )['matches']
        for i in check:
            if(i["message"][0:27]=="Possible spelling mistake"):
                print(i["message"] + " in line "+str(c1))
        c1=c1+1

##############################################################################################################
        
        
#function for finding similarity
def similarity(X,Y):
        X=str(X)
        # Program to measure similarity between  
        # two sentences using cosine similarity. 
        X=str(X)
        Y=str(Y)

        X = X.lower() 
        Y = Y.lower() 

        # tokenization 
        X_list = word_tokenize(X)  
        Y_list = word_tokenize(Y) 

        # sw contains the list of stopwords 
        sw = stopwords.words('english')  
        l1 =[];l2 =[] 

        # remove stop words from string 
        X_set = {w for w in X_list if not w in sw}  
        Y_set = {w for w in Y_list if not w in sw} 

        # form a set containing keywords of both strings  
        rvector = X_set.union(Y_set)  
        for w in rvector: 
            if w in X_set: l1.append(1) # create a vector 
            else: l1.append(0) 
            if w in Y_set: l2.append(1) 
            else: l2.append(0) 
        c = 0

        # cosine formula  
        for i in range(len(rvector)): 
            c+= l1[i]*l2[i] 

        if(float((sum(l1)*sum(l2))**0.5) ==0.0):
            return(0)
        else:
            cosine = c / float((sum(l1)*sum(l2))**0.5)
            return cosine
        
##############################################################################################################

#download the edx dataset
edx=pd.read_csv("EDX_data.csv")

##############################################################################################################

#function for retrieving text from html scripts
def data_cleaning(data,empty_list):
    for doc in data:
        #html=strip_tags(str(doc))
        html=BeautifulSoup(str(doc), 'lxml').get_text()
        dig=re.sub(" \d+", " ", html)
        sp_char=re.sub('[^A-Za-z0-9]+', ' ', dig)
        empty_list.append(sp_char)
        
        
#Downloading our database courses from location
df=pd.read_csv("Courses.csv")

##############################################################################################################

#Formation of our main dataset

df1=pd.DataFrame()
df1["course title"]=df["course_title"]
df1["course description"]=df["course_description"]
df1["course subtitle"]=df["course_subtitle"]
df1["course id"]=df["course_id"]



#########################################################################
#Extract and take the course id as input and extract the course description and title for only that course for validation with ideal
id="47215856678036"
course_description=""
course_title=""
###########################################################################



#Cleaning the dataset
c=0
for i in df1["course id"]:
    if(id==i):
        course_description = df1["clean course description"][c]
        course_title = df1["course title"][c]
    c=c+1
    
############################################################################################################   
    
#Validation checking caller module
c=0
flag=0
for i in edx["Name"]:
    if(similarity(course_title,i)>0.45 and similarity(course_description,edx["Description"][c])>0.45):
        tttt=0
    elif(similarity(course_title,i)>0.45 and similarity(course_description,edx["Description"][c])<0.45):
        flag=1
        print("some changes are required. For demo: ")
        print(edx["Description"][c])
if(flag == 0):
    print("No changes required!!")    
    
##############################################################################################################
    
#Calling spell_gram check 

for i in df1["course_description"]:
    grammer_spelling_check(i)
    print("\n")


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
